## 범주형 데이터의 검정
- 기본 원리: 카이제곱-검정
    - 관측 도수: 실제 관측됨 도수
    - 기대 도수: 특정한 분포나 독립성을 따른다는 가정에서 나올 수 있는 도수

- 검정
    - p-value < 유의수준 -> 관측도수와 기대도수가 차이있음 -> 상관관계 있음
    - p-value>= 유의수준 -> 관측도수와 기대도수가 차이가 없음 -> 상관관계 없음

### 1. 적합성 검정
- 하나의 변수(요인)에 대하여 범주(수준)별 비율이 특정 분포(비율)을 따르는 지 검정하는 것
- 적합성 검정을 하기 위한 조건
    - 하나의 모집단에 상호 배타적인 여러 범주가 있는 데이터
    - 데이터는 랜덤 표집으로 되어야 함
    - 총 빈도수가 충분히 크고(>50), 각 수준별 5개 이상의 기대도수 필요
        - 불만족 시, 제 1종 오류와 제 2종 오류가 증가
가설
- 귀무가설: 관측도수와 기대도수가 차이가 없다.
- 대립가설: 관측도수와 기대도수가 차이가 있다.

데이터수집
- 하나의 변수에 대하여 수준별 관측도수(범주별 데이터 개수) 수집

예시
- 상황
    - 고혈압 환자 100명 치료 방법 선호도 조사. 예상 선호도는 약물치료(A) 50%, 식이요법(B) 30%, 운동요법(C) 20%이다. 선호도 조사결과가 다음과 같을 때, 예상 선호도와 차이가 있는 지 검정
- 데이터수집
    - 약물치료(A) 60명, 식이요법(B) 25명, 운동요법(C) 15명

In [2]:
from scipy.stats import chisquare
#유의수준
alpha=0.05

#관측도수
observed=[60,25,15]

#기대도수. 각 치료 방법에 대한 선호도 예상 비율
total_patient=sum(observed)
expected_ratios=[0.5,0.3,0.2]
expected=[total_patient*ratio for ratio in expected_ratios]

#적합성 검정 수행
chi2_stat, p_value= chisquare(f_obs=observed, f_exp=expected)
print(f'카이제곱 통계량: {chi2_stat:.4f}')
print(f'p_value: {p_value:.4f}')
if p_value <0.05:
    print('귀무가설 기각: 관측된 데이터는 예상 비율과 일치하지 않음')
else:
    print('귀무가설 채택: 관측된 데이터는 예상 비율과 일치함')

카이제곱 통계량: 4.0833
p_value: 0.1298
귀무가설 채택: 관측된 데이터는 예상 비율과 일치함


### 2. 동질성 검정
- 범주를 가진 요인이 1개이고 부-모집단이 여러 개일 때 모든 부-모집단이 동일한 분포를 하고 있는 지 검정
- 동질성 검정을 하기 위한 조건
    - 여러 개의 부-모집단에 대해 하나의 변수(요인)의 범주(수준)별 비율이 동일한 지 검정(서로 배타적인 범주여야 함)
    - 각 부-모집단에 대해 무작위 표집을 하여야 함
    - 총빈도수가 충분히 크고(>50), 각 수준별 5개 이상의 기대도수 필요
        - 불만족 시, 제1종 오류와 제2종 오류가 증가
가설
- 귀무가설: sub-group간 요인의 수준별 비율 차이가 없다.(모든 sub-group이 동일 분포이다.)
- 대립가설: sub-group간 요인의 수준별 비율 차이가 있는 것이 하나라도 있다.

데이터 수집
- 각 부-모집단별로 정해진 표본의 크기만큼 수집

예시

- 상황
    - 골절환자 200명에 대하여 2개의 다른 치료방법(A,B)의 치료 결과에 대한 조사. 치료 결과는 완치, 부분완치, 실패로 구분하였다. 두 병원의 치료결과에 차이가 있는지 검정

- 데이터수집
    - 방법A: 완치 50명, 부분완치 30명, 실패 20명
    - 방법B: 완치 45명, 부분완치 35명, 실패 20명

In [3]:
import pandas as pd
from scipy.stats import chi2_contingency

data={
    '방법': ['방법A','방법A','방법A','방법B','방법B','방법B'],
    '치료결과':['완치', '부분완치','실패','완치', '부분완치','실패'],
    '빈도':[50,30,20,45,35,20]
}
df= pd.DataFrame(data)
df

,방법,치료결과,빈도
0,방법A,완치,50
1,방법A,부분완치,30
2,방법A,실패,20
3,방법B,완치,45
4,방법B,부분완치,35
5,방법B,실패,20


In [4]:
#교차표 생성
cross_tab=pd.pivot_table(
    df,
    index='방법',
    columns='치료결과',
    values='빈도',
    aggfunc='sum'  #집계함수
    #margins=True,
    #margins_name='합계'
)
print('교차표(치료 방법별 치료 결과):')
print(cross_tab)

교차표(치료 방법별 치료 결과):
치료결과  부분완치  실패  완치
방법                
방법A     30  20  50
방법B     35  20  45


In [5]:
#동질성 검정 수행
chi2, p_value, dof, expected=chi2_contingency(cross_tab)

print(f'기대도수:\n{expected}', end='\n\n')
print('카이제곱 검정 결과')
print(f'카이제곱 통계량: {chi2:.4f}')
print(f'p_value: {p_value:.4f}')
print(f'자유도: {dof}')

기대도수:
[[32.5 20.  47.5]
 [32.5 20.  47.5]]

카이제곱 검정 결과
카이제곱 통계량: 0.6478
p_value: 0.7233
자유도: 2


In [6]:
if p_value<alpha:
    print('귀무가설 기각: 두 치료 방법의 치료 결과 분포는 동일하지 않음')
else:
    print('귀무가설 채택: 두 치료 방법의 치료 결과 분포는 동일함')

귀무가설 채택: 두 치료 방법의 치료 결과 분포는 동일함


### 3. 독립성 검정 = 교차분석
- 2개의 요인이 서로 상관관계가 없는 지 검정으로서 교차분석이라고도 함
- 독립성 검정을 하기 위한 조건
    - 하나의 모집단에 2개의 요인에 대하여 상호 배타적인 여러 수준이 있는 데이터
    - 데이터는 랜덤 표집으로 수집되어야 함
    - 총빈도수가 충분히 크고(>50), 각 수준별 5개 이상의 기대도수 필요
        - 불만족 시, 제1종 오류와 제2종 오류가 증가

가설
- 귀무가설: 두 요인 변수는 서로 독립적이다(상관관계가 없다).
- 대립가설: 두 요인 변수는 서로 독립적이지 않다(상관관계가 있다).

데이터 수집
- 두 변수의 각 수준 조합에 대해 빈도수 수집

예시
- 상황
    - 흡연 여부와 폐암 발생 여부에 대한 조사 실시. 흡연 여부는 비흡연, 흡연으로 구분하고 폐암 발생 여부는 미발생, 발생으로 구분
- 데이터 수집
    - 비흡연-미발생 50명, 비흡연-발생 10명, 흡연-미발생 20명, 흡연-발생 20명

In [7]:
import pandas as pd
from scipy.stats import chi2_contingency

data={
    '흡연여부': ['비흡연','비흡연','흡연','흡연'],
    '질병발생':['미발생','발생','미발생','발생'],
    '빈도':[50,10,20,20]
}
df=pd.DataFrame(data)
df

,흡연여부,질병발생,빈도
0,비흡연,미발생,50
1,비흡연,발생,10
2,흡연,미발생,20
3,흡연,발생,20


In [8]:
cross_tab=pd.pivot_table(
    df,
    index='흡연여부',
    columns='질병발생',
    values='빈도',
    aggfunc='sum'
)
print('교차표(흡연 여부와 질병 발생): ')
display(cross_tab)

#독립성 검정 수행
chi2,p_value,dof,expected=chi2_contingency(cross_tab)
print(f'기대도수:\n{expected}', end='\n\n')
print('카이제곱 검정 결과')
print(f'카이제곱 통계량: {chi2:.4f}')
print(f'p_value: {p_value:.4f}')
print(f'자유도: {dof}\n')

#결론 도출
alpha=0.05
if p_value<alpha:
    print('귀무가설 기각: 흡연 여부와 질병 발생은 독립적이지 않음(상관관계가 있음)')
else:
    print('귀무가설 채택: 흡연 여부와 질병 발생은 독립적임(상관관계가 없음)')


교차표(흡연 여부와 질병 발생): 


질병발생,미발생,발생
흡연여부,,
비흡연,50,10
흡연,20,20


기대도수:
[[42. 18.]
 [28. 12.]]

카이제곱 검정 결과
카이제곱 통계량: 11.1607
p_value: 0.0008
자유도: 1

귀무가설 기각: 흡연 여부와 질병 발생은 독립적이지 않음(상관관계가 있음)


## 카이제곱 검정의 보정
카이제곱 검정에서 보정이 필요한 이유
- 기대도수가 5 이하의 셀이 20%를 넘는 경우 카이제곱 검정은 검정력이 저하됨
    - 카이제곱 검정은 교차표에 나타나는 빈도의 비연속형 확률분포가 연속형 분포임 카이제곱 분포를 근사적으로 따른다고 가정하기 때문
    - 표본의 크기가 충분히 크지 않으면, 카이제곱 분포로의 근사화가 잘 되지 않아 오류가 증가     

피셔의 정확검정을 이용한 카이제곱 검정의 보정
- 피셔의 정확검정은 초기하분포 기반의 범주형 데이터의 독립성 검정 방법
    - 카이제곱 검정은 이항분포 기반
- 주변 빈도가 고정된 상태에서 발생 가능한 모든 조합에 해당하는 자료의 발생 확률을 계산
- 표본의 크기가 너무 작거나, 범주가 너무 많아서 범주별 도수가 극도로 작아지는 상황에 주로 사용      

피셔의 정확검정 예시
- 위와 동일한 예시에서 관측도수가 1/10일 때, 카이제곱 검정을 적용한 경우

In [9]:
import pandas as pd
from scipy.stats import chi2_contingency

data={
    '흡연여부': ['비흡연','비흡연','흡연','흡연'],
    '질병발생':['미발생','발생','미발생','발생'],
    '빈도':[5,1,2,2]
}
df=pd.DataFrame(data)

cross_tab=pd.pivot_table(
    df,
    index='흡연여부',
    columns='질병발생',
    values='빈도',
    aggfunc='sum'
)
print('교차표(흡연 여부와 질병 발생): ')
display(cross_tab)

#독립성 검정 수행
chi2,p_value,dof,expected=chi2_contingency(cross_tab)
print(f'기대도수:\n{expected}', end='\n\n')
print('카이제곱 검정 결과')
print(f'카이제곱 통계량: {chi2:.4f}')
print(f'p_value: {p_value:.4f}')
print(f'자유도: {dof}\n')

#결론 도출
alpha=0.05
if p_value<alpha:
    print('귀무가설 기각: 흡연 여부와 질병 발생은 독립적이지 않음(상관관계가 있음)')
else:
    print('귀무가설 채택: 흡연 여부와 질병 발생은 독립적임(상관관계가 없음)')


교차표(흡연 여부와 질병 발생): 


질병발생,미발생,발생
흡연여부,,
비흡연,5,1
흡연,2,2


기대도수:
[[4.2 1.8]
 [2.8 1.2]]

카이제곱 검정 결과
카이제곱 통계량: 0.1786
p_value: 0.6726
자유도: 1

귀무가설 채택: 흡연 여부와 질병 발생은 독립적임(상관관계가 없음)


빈도수가 작아졌기에 귀무가설이 채택됨

- 위와 동일한 예시에서 관측도수가 1/10일 때, 피셔의 정확검정을 적용한 경우

In [10]:
from scipy.stats import fisher_exact

data={
    '흡연여부': ['비흡연','비흡연','흡연','흡연'],
    '질병발생':['미발생','발생','미발생','발생'],
    '빈도':[5,1,2,2]

}
df=pd.DataFrame(data)

cross_tab=pd.pivot_table(
    df,
    index='흡연여부',
    columns='질병발생',
    values='빈도',
    aggfunc='sum'
)
print('교차표(흡연 여부와 질병 발생): ')
display(cross_tab)

#독립성 검정 수행
stat,p_value=fisher_exact(cross_tab)
print(f'기대도수:\n{expected}', end='\n\n')
print('피셔의 정확 검정 결과')
print(f'검정통계량: {stat:.4f}')
print(f'p_value: {p_value:.4f}')

#결론 도출
alpha=0.05
if p_value<alpha:
    print('귀무가설 기각: 흡연 여부와 질병 발생은 독립적이지 않음(상관관계가 있음)')
else:
    print('귀무가설 채택: 흡연 여부와 질병 발생은 독립적임(상관관계가 없음)')


교차표(흡연 여부와 질병 발생): 


질병발생,미발생,발생
흡연여부,,
비흡연,5,1
흡연,2,2


기대도수:
[[4.2 1.8]
 [2.8 1.2]]

피셔의 정확 검정 결과
검정통계량: 5.0000
p_value: 0.5000
귀무가설 채택: 흡연 여부와 질병 발생은 독립적임(상관관계가 없음)


귀무가설이 기각되지는 않았지만 검정통계량이 증가하고 p-value가 더 작게 나온 것을 확인할 수 있음 > 연구가설 채택 가능성에 근접함 > 표본이 작을 때 보정해주는 역할을 해 줌

## 범주형 데이터 분석 실습

데이터 수집
- 3개 변수에 대한 592개의 관측치를 교차 분석하여 얻은 3차원 배열
- 변수
    - Hair: 머리 색상. Black/Brown/Red/Blond
    - Eye: 눈 색상. Brown/Blue/Green/Hazel
    - Sex: 성별. Male/Female
    - Freq: 빈도수.

In [11]:
import pandas as pd
import statsmodels.api as sm

#데이터셋 로드
df=sm.datasets.get_rdataset('HairEyeColor','datasets').data

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Hair    32 non-null     object
 1   Eye     32 non-null     object
 2   Sex     32 non-null     object
 3   Freq    32 non-null     int64 
dtypes: int64(1), object(3)
memory usage: 1.1+ KB


In [12]:
df.head()

,Hair,Eye,Sex,Freq
0,Black,Brown,Male,32
1,Brown,Brown,Male,53
2,Red,Brown,Male,10
3,Blond,Brown,Male,3
4,Black,Blue,Male,11


### 1. 적합성 검정
- 머리 색상의 분포가 1:1:1:1 비율을 따르는 지 검정

In [13]:
from scipy.stats import chisquare,chi2_contingency

alpha=0.05

cross_tab=pd.pivot_table(
    df,
    values='Freq',
    columns='Hair',
    aggfunc='sum',
    fill_value=0 #결측값을 0으로 채움
)

print('교차표(Hair):')
display(cross_tab)

교차표(Hair):


Hair,Black,Blond,Brown,Red
Freq,108,127,286,71


In [14]:
total=cross_tab.loc['Freq'].sum()

#예상비율
expected_ratios=[0.25,0.25,0.25,0.25]
expected=[total*ratio for ratio in expected_ratios]

#적합성 검정 수행
chi2_stat,p_value=chisquare(f_obs=cross_tab.loc['Freq'], f_exp=expected)
print(f'카이제곱 통계량: {chi2_stat:.4f}')
print(f'p_value: {p_value:.4f}')
if p_value <0.05:
    print('귀무가설 기각: 머리 색상은 예상 비율과 일치하지 않음')
else:
    print('귀무가설 채택: 머리 색상은 예상 비율과 일치함')


카이제곱 통계량: 182.5270
p_value: 0.0000
귀무가설 기각: 머리 색상은 예상 비율과 일치하지 않음
